In [31]:
import pandas as pd

from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 90%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""))

# pd.set_option('max_colwidth', 100)


In [3]:
mf = pd.read_csv('mf_buildstock.csv', index_col='Building')

In [32]:
mf.head()

,Location Weather Year,Usage Level,Geometry Building Type ACS,Location,Location Region,Location Weather Filename,Vintage ACS,Vintage,Heating Fuel,Geometry Building Type RECS,...,Misc Well Pump,Ducts,Infiltration,Natural Ventilation,Mechanical Ventilation,Bathroom Spot Vent Hour,Range Spot Vent Hour,PV,Days Shifted,Geometry Perimeter Footprint Ratio
Building,,,,,,,,,,,,,,,,,,,,,
1,TMY3,Medium,20 to 49 Unit,CA_Los.Angeles.Intl.AP.722950,CR11,USA_CA_Los.Angeles.Intl.AP.722950_TMY3.epw,1940-59s,1950s,Natural Gas,Multi-Family with 5+ Units,...,None,"20% Leakage, R-4",20 ACH50,"Year-Round, 3 days/wk",None,Hour6,Hour14,None,Day276,0.13
2,TMY3,High,5 to 9 Unit,IN_Indianapolis.Intl.AP.724380,CR04,USA_IN_Indianapolis.Intl.AP.724380_TMY3.epw,1980-99s,1980s,Electricity,Multi-Family with 5+ Units,...,Typical Efficiency,"10% Leakage, Uninsulated",10 ACH50,"Year-Round, 3 days/wk",None,Hour6,Hour22,None,Day245,0.13
3,TMY3,Medium,5 to 9 Unit,TX_Houston-Bush.Intercontinental.AP.722430,CR09,USA_TX_Houston-Bush.Intercontinental.AP.722430_TMY3.epw,1960-79s,1960s,Electricity,Multi-Family with 5+ Units,...,Typical Efficiency,"20% Leakage, R-4",20 ACH50,"Year-Round, 3 days/wk",None,Hour19,Hour10,None,Day1,0.13
4,TMY3,Medium,20 to 49 Unit,NJ_Newark.Intl.AP.725020,CR07,USA_NJ_Newark.Intl.AP.725020_TMY3.epw,1960-79s,1960s,Natural Gas,Multi-Family with 5+ Units,...,None,"30% Leakage, Uninsulated",15 ACH50,"Year-Round, 3 days/wk",None,Hour7,Hour18,None,Day29,0.13
5,TMY3,Medium,2 Unit,PA_Pittsburgh.Intl.AP.725200,CR07,USA_PA_Pittsburgh.Intl.AP.725200_TMY3.epw,<1940,<1940,Natural Gas,Multi-Family with 2 - 4 Units,...,None,"30% Leakage, Uninsulated",20 ACH50,"Year-Round, 3 days/wk",None,Hour11,Hour17,None,Day215,0.13


In [8]:
dirs = ['North', 'Northeast', 'East', 'Southeast', 'South', 'Southwest', 'West', 'Northwest']
inv_dirs = ['South', 'Southwest', 'West', 'Northwest', 'North', 'Northeast', 'East', 'Southeast']
orient_dict = dict(zip(dirs, inv_dirs))

In [10]:
master_df = pd.DataFrame()
for idx in mf.index:
    bldg = mf.loc[idx]
    units = int(bldg['Geometry Building Number Units MF'])
    floors = int(bldg['Geometry Stories'])
    num_units_per_floor = units/floors

    bstock = pd.DataFrame(columns= mf.columns, index=range(int(1),units+1))
    bstock.index.name = 'Building'
#     print("ID: ", idx)
#     print("floors: ", floors)
#     print("units: ", units)

    front_orient = bldg['Orientation']
    rear_orient = front_orient
    num_middle_z = floors - 2
    if num_units_per_floor%2 != 0: #no rear
        has_rear_units = False
        num_middle_x = int(num_units_per_floor - 2)
    else: #rear units
        has_rear_units = True
        rear_orient = orient_dict[bldg['Orientation']]
        num_middle_x = int((num_units_per_floor/2) - 2)   
    if num_middle_x < 0:
        num_middle_x = 0

    if has_rear_units:
        num_units_front = int(units/2)
        units_add = int(num_units_per_floor/2)
    else:
        num_units_front = int(units)
        units_add = int(num_units_per_floor)

    #Initial horizontal locations
    h_left = [1]
    if num_middle_x > 0:
        h_mid = list(range(2, 2+int(num_middle_x)))
    else:
        h_mid = []
    if num_units_front/floors > 1:
        h_right = [num_middle_x + 2]
    else:
        h_right = []
    #Define horizontal locatons by floor
    for f in range(2,floors+1):
        h_left += [h_left[-1] + units_add]
        h_mid += list(map(lambda x: x+units_add, h_mid[-num_middle_x:]))
        h_right += list(map(lambda x: x+units_add, h_right[-1:]))

    #LEVELS
    #..bottom
    if num_units_front/floors > 1:
        lv_bottom = list(range(1,2+num_middle_x+1))
    else:
        lv_bottom = [1]
    #..middle
    if num_middle_z > 0:
        lv_middle = list(range(lv_bottom[-1]+1, lv_bottom[-1]+1 + (units_add*num_middle_z)))
    else:
        lv_middle = []
    #..top
    if floors > 1:
        lv_top = list(range(units_add*(floors-1) + 1, units_add*floors +1))
    else:
        lv_top = []

    if has_rear_units:
        h_left += list(map(lambda x:x+num_units_front, h_left))
        h_mid += list(map(lambda x:x+num_units_front, h_mid))
        h_right += list(map(lambda x:x+num_units_front, h_right))
        lv_bottom += list(map(lambda x:x+num_units_front, lv_bottom))
        lv_middle += list(map(lambda x:x+num_units_front, lv_middle))
        lv_top += list(map(lambda x:x+num_units_front, lv_top))

    floor_unit = 1
    for unit in range(1,units+1):     
        bstock.loc[unit] = bldg

        if unit in h_left:
            bstock.loc[unit, 'Geometry Building Horizontal Location MF'] = 'Left'
        elif unit in h_mid:
            bstock.loc[unit, 'Geometry Building Horizontal Location MF'] = 'Middle'
        elif unit in h_right:
            bstock.loc[unit, 'Geometry Building Horizontal Location MF'] = 'Right'

        if unit in lv_bottom:
            bstock.loc[unit, 'Geometry Building Level MF'] = 'Bottom'
        elif unit in lv_middle:
            bstock.loc[unit, 'Geometry Building Level MF'] = 'Middle'
        elif unit in lv_top:
            bstock.loc[unit, 'Geometry Building Level MF'] = 'Top'

        if unit <= units/2:
            bstock.loc[unit, 'Orientation'] = front_orient
        else:
            bstock.loc[unit, 'Orientation'] = rear_orient
            
#         bstock.loc[unit, 'Geometry Single Unit Has Rear Units'] = has_rear_units
    bstock['building_code'] = 'bldg_{}'.format(idx)
    if master_df.empty:
        master_df = bstock.copy()
    else:
        master_df = master_df.append(bstock)
#     bstock.to_csv('./mf_buildstock_singles/buildstock_' + str(idx) + '.csv')
master_df.index= range(1,len(master_df)+1)
master_df.index.rename('Building', inplace=True)

In [12]:
bldg_map = pd.DataFrame(columns=['building_range'], index=master_df.building_code.unique())
for bldg in master_df.building_code.unique():
    idx = list(master_df[master_df.building_code == bldg].index.values)
    bldg_map.loc[bldg]['building_range'] = '"{}"'.format(str(idx).replace(' ','').strip(']['))
bldg_map.head()
bldg_map.to_csv('./building_id_mapping.csv')

In [33]:
bldg_map

,building_range
bldg_1,"""1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24"""
bldg_2,"""25,26,27,28,29,30"""
bldg_3,"""31,32,33,34,35,36,37,38"""
bldg_4,"""39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62"""
bldg_5,"""63,64"""
bldg_6,"""65,66,67,68"""
bldg_7,"""69,70,71"""
bldg_8,"""72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102..."
bldg_9,"""114,115,116"""
bldg_10,"""117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140..."


In [34]:
master_df = master_df.drop('building_code', axis=1)
master_df.to_csv('buildstock_singleunits.csv')

In [35]:
master_df.head()

,Location Weather Year,Usage Level,Geometry Building Type ACS,Location,Location Region,Location Weather Filename,Vintage ACS,Vintage,Heating Fuel,Geometry Building Type RECS,...,Infiltration,Natural Ventilation,Mechanical Ventilation,Bathroom Spot Vent Hour,Range Spot Vent Hour,PV,Days Shifted,Geometry Perimeter Footprint Ratio,Geometry Building Horizontal Location MF,Geometry Building Level MF
Building,,,,,,,,,,,,,,,,,,,,,
1,TMY3,Medium,20 to 49 Unit,CA_Los.Angeles.Intl.AP.722950,CR11,USA_CA_Los.Angeles.Intl.AP.722950_TMY3.epw,1940-59s,1950s,Natural Gas,Multi-Family with 5+ Units,...,20 ACH50,"Year-Round, 3 days/wk",None,Hour6,Hour14,None,Day276,0.13,Left,Bottom
2,TMY3,Medium,20 to 49 Unit,CA_Los.Angeles.Intl.AP.722950,CR11,USA_CA_Los.Angeles.Intl.AP.722950_TMY3.epw,1940-59s,1950s,Natural Gas,Multi-Family with 5+ Units,...,20 ACH50,"Year-Round, 3 days/wk",None,Hour6,Hour14,None,Day276,0.13,Middle,Bottom
3,TMY3,Medium,20 to 49 Unit,CA_Los.Angeles.Intl.AP.722950,CR11,USA_CA_Los.Angeles.Intl.AP.722950_TMY3.epw,1940-59s,1950s,Natural Gas,Multi-Family with 5+ Units,...,20 ACH50,"Year-Round, 3 days/wk",None,Hour6,Hour14,None,Day276,0.13,Middle,Bottom
4,TMY3,Medium,20 to 49 Unit,CA_Los.Angeles.Intl.AP.722950,CR11,USA_CA_Los.Angeles.Intl.AP.722950_TMY3.epw,1940-59s,1950s,Natural Gas,Multi-Family with 5+ Units,...,20 ACH50,"Year-Round, 3 days/wk",None,Hour6,Hour14,None,Day276,0.13,Middle,Bottom
5,TMY3,Medium,20 to 49 Unit,CA_Los.Angeles.Intl.AP.722950,CR11,USA_CA_Los.Angeles.Intl.AP.722950_TMY3.epw,1940-59s,1950s,Natural Gas,Multi-Family with 5+ Units,...,20 ACH50,"Year-Round, 3 days/wk",None,Hour6,Hour14,None,Day276,0.13,Middle,Bottom


In [41]:
bstock.filter(['Geometry Building Level MF', 'Geometry Building Horizontal Location MF', 'Orientation', 'Geometry Stories', 'Geometry Building Number Units MF'])

,Geometry Building Level MF,Geometry Building Horizontal Location MF,Orientation,Geometry Stories,Geometry Building Number Units MF
Building,,,,,
1,Bottom,Left,North,1,4
2,Bottom,Right,North,1,4
3,Bottom,Left,South,1,4
4,Bottom,Right,South,1,4
